# Introduction

In Part 2 of this project, I will examine the performance of New York City high school student's on the SAT examine to determine if there are any demographic factors that might influence student performance. However, in Part 1 of this project, I will simply clean the eight datasets below before combining them into a single dataset.

The SAT exam is a placement test that American universities use in order to help determine which which students to accept. The SAT has three sections, each of which is worth a maximum of 800 points. A high school's average SAT score is also often used as an indicator of the quality of the school's curriculum.

In my analysis, I will be combining seven datasets into a single, cleaned pandas dataframe. All eight datasets pertain to schools in New York City.
- [ap_2010.csv](https://data.cityofnewyork.us/api/views/itfs-ms3e/files/cfb55d3e-e18d-4d78-982d-5f892d72f28c?download=true&filename=AP%20Results.xlsx) = shows Advanced Placement (AP) exam results (which are used to help universities find qualified applicants) for each high school.
- [class_size.csv](https://data.cityofnewyork.us/api/views/urz7-pzb3/files/0fd9def3-28bd-484a-8977-7f30d142ac9f?download=true&filename=2010-2011\_Class\_Size\_School\_Level\_Detail.xlsx) dataset contains information on the size of the classes within a school.
- [demographics.csv](https://data.cityofnewyork.us/api/views/ihfw-zy9j/files/a15740cc-a4ae-4478-88a9-4ecc677b6949?download=true&filename=School%20Demographics%20and%20Accountability%20Snapshot%202006-2012.xlsx) = contains demographic information for each school.
- [graduation.csv](https://data.cityofnewyork.us/api/views/vh2h-md7a/files/a06dc3fd-d93a-4050-9e3f-077c0c430b10?download=true&filename=Graduation%20Outcomes.xlsx) = contains the percentage of students who graduated, as well as other graduation outcome information.
- [hs_directory.csv](https://data.cityofnewyork.us/api/views/n3p6-zve2/files/416d8b2c-7270-436d-84d9-f76a28982416?download=true&filename=2014-2015\_DOE\_High\_Shool\_Directory.xlsx) = contains attendance information for each school.
- [sat_results.csv](https://data.cityofnewyork.us/api/views/f9bf-2cp4/files/7e5d7800-8b8d-458d-a4d7-0d9b30ca53e5?download=true&filename=SAT%20Results.xlsx) = contains SAT scores for each high school.
- [survey_all.txt](https://data.cityofnewyork.us/api/views/mnz3-dyi8/files/aa68d821-4dbb-4eb2-9448-3d8cbbad5044?download=true&filename=Survey%20Data%20Dictionary.xls) = contains surveys of parents, teachers, and students at each school.
- [survey_d75.txt](https://data.cityofnewyork.us/api/views/mnz3-dyi8/files/aa68d821-4dbb-4eb2-9448-3d8cbbad5044?download=true&filename=Survey%20Data%20Dictionary.xls) = contains surveys of parents, teachers, and students at each school in NYC district 75.

The SAT exam is a placement test that American universities use in order to help determine which which students to accept. The SAT has three sections, each of which is worth a maximum of 800 points. A high school's average SAT score is also often used as an indicator of the quality of the school's curriculum.

# First Look at the Datasets

Since I am planing on combining all eight datasets, I will have to spend a significant amount of time cleaning the data prior to analyzing it. First, I will import the appropriate libraries we will be using throughout our analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

## Reading in the Intial Datasets

In order to make referencing the various datasets easier, I will combine all six of the csv files into a single dictionary (called data) and combined the two surveys into a single txt file.

In [2]:
data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

# Adds all files in data_files list (located in the schools folder) to the data dictionary
for f in data_files:
    d = pd.read_csv("schools/{0}".format(f))
    data[f.replace(".csv", "")] = d # Removes the .csv from the name and makes file name a key in the dictionary

data.keys()

dict_keys(['ap_2010', 'class_size', 'demographics', 'graduation', 'hs_directory', 'sat_results'])

Now that we have combined all of the csv files into a single, easily-accessible dictionary, let's concatonate the txt files (all_survey and d75_survey) into a single file.

In [3]:
# Read in the all_survey and d75_survey data and concatonates them into a single dataset
all_survey = pd.read_csv("schools/survey_all.txt", delimiter="\t", encoding="windows-1252")
d75_survey = pd.read_csv("schools/survey_d75.txt", delimiter="\t", encoding="windows-1252")

survey = pd.concat([all_survey, d75_survey], axis=0)

survey.columns

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


Index(['N_p', 'N_s', 'N_t', 'aca_p_11', 'aca_s_11', 'aca_t_11', 'aca_tot_11',
       'bn', 'com_p_11', 'com_s_11',
       ...
       't_q8c_1', 't_q8c_2', 't_q8c_3', 't_q8c_4', 't_q9', 't_q9_1', 't_q9_2',
       't_q9_3', 't_q9_4', 't_q9_5'],
      dtype='object', length=2773)

Now that I can easily reference the various datasets, I will remove various columns from my dataset so that it only contains those that will be useful for my analysis.

## Limiting the Number of Columns in the Survey Dataset

The survey dataset has 2773 columns, which is obviously much more than will be useful for my analysis. According to the GitHub website where I originally downloaded the two survey datasets, the most important columns in this database are the columnsin the survey_fields list below. Thus, I can get rid of all other columns in the survey_fields dataset and then add the survey information to the data dictionary.

In [4]:
# List of useful columns
survey_fields = [
                "DBN", "rr_s", "rr_t", "rr_p",
                "N_s", "N_t", "N_p", "saf_p_11",
                "com_p_11", "eng_p_11", "aca_p_11",
                "saf_t_11", "com_t_11", "eng_t_11",
                "aca_t_11", "saf_s_11", "com_s_11",
                "eng_s_11", "aca_s_11", "saf_tot_11",
                "com_tot_11", "eng_tot_11", "aca_tot_11"
                ]

survey = survey.rename(columns={"dbn": "DBN"}) # capitalizes the "dbn" column name
survey = survey.loc[:, survey_fields] # drops columns from the survey dataset that aren't in useful columns list
data["survey"] = survey # saves new survey dataset to the data dictionary
data["survey"].columns

Index(['DBN', 'rr_s', 'rr_t', 'rr_p', 'N_s', 'N_t', 'N_p', 'saf_p_11',
       'com_p_11', 'eng_p_11', 'aca_p_11', 'saf_t_11', 'com_t_11', 'eng_t_11',
       'aca_t_11', 'saf_s_11', 'com_s_11', 'eng_s_11', 'aca_s_11',
       'saf_tot_11', 'com_tot_11', 'eng_tot_11', 'aca_tot_11'],
      dtype='object')

## Fixing DBN Columns

I will need to edit the DBN columns in two of the data datasets via the following steps:
- hs_directory = DBN column needs to be capitalized
- class_size = DBN column needs to be created using the CSD and SCHOOL CODE columns

In [5]:
data["hs_directory"] = data["hs_directory"].rename(columns={"dbn":"DBN"}) # capitalizes "dbn" column name


Next, I will create a DBN column in the class_size directory that is similar to the one in the hs_directory. As seen below, the DBN column appears to be a combination of the CSD and SCHOOL CODE columns, so I will take a look at what the DBN column is supposed to look like (by looking at it in the hs_directory) and then look at the CSD and SCHOOL CODE columns in class_size.

In [6]:
data["hs_directory"]["DBN"].value_counts().head()

29Q498    1
30Q445    1
15K463    1
28Q690    1
14K449    1
Name: DBN, dtype: int64

In [7]:
data["class_size"]["CSD"].value_counts().head(5)

2     1985
10    1568
9     1268
31    1212
11    1175
Name: CSD, dtype: int64

In [8]:
data["class_size"]["SCHOOL CODE"].value_counts().head(5)

K429    57
X505    56
X517    56
K448    52
K690    52
Name: SCHOOL CODE, dtype: int64

The DBN column is basically the CSD column (which represents each district) followed by the SCHOOL CODE column (which represents each school). Therefore, I should be able to combine the two together to make a DBN column in class_size.

Note that the districts in the CSD column are either one or two-digit numbers, obviously dependant on the numbers' size. However, all of the DBN's are six digit (the last four being the school code), implying that all one-digit districts are padded by a zero. To make all of the values in my new DBN column in the class_size dataset six digits, I will create a simple function that pads all one-digit numbers with a zero to make them two-digit numbers.

In [9]:
def pad_csd(num):
    string_representation = str(num)
    if len(string_representation) > 1:
        return string_representation
    else:
        return "0" + string_representation

In [10]:
data["class_size"]["CSD"] = data["class_size"]["CSD"].apply(pad_csd)
data["class_size"]["CSD"].value_counts().head()

02    1985
10    1568
09    1268
31    1212
11    1175
Name: CSD, dtype: int64

Next, I will combined the CSD column values to the SCHOOL CODE column values to create a new DBN column in the class_size dataset.

In [11]:
data["class_size"]["DBN"] = data["class_size"]["CSD"] + data["class_size"]["SCHOOL CODE"]
data["class_size"]["DBN"].value_counts().head(5)

15K429    57
09X517    56
09X505    56
21K690    52
15K448    52
Name: DBN, dtype: int64

Now that the DBN columns have been fixed for both datasets, I will start working on preparing the data so that it can eventually be merged.

# Preparing the Data for Combination

In this section, I will make a series of changes to the dataset that will make it easier to analyze. First, I will create some new columns that contain useful information to my analysis:
- SAT Score = the sum of the three categorical SAT scores (Math, CR, Writing)
- Lon, Lat = show the longitude and latitude of the school (useful for mapping later)

Afterwards, I will remove unnecessary information from a few of the datasets, which will make the datasets easier to combine as they will eventually all contain roughly the same number of data entries. The datasets I will modify include:
- Class Size = reduce the dataset to a subset containing only high school programs, general ed programs, and reducing the dataset by finding the mean of all DBN's per school
- Demographics = I will select only information from the most recent year, which is the 2011-2012 school year
- Graduation = will select only students that graduated in 2006, which is the most recent year

After these steps have been completed, we can combine all of the datasets in the dictionary into a single dataset.

## Creating New Columns

First, I will take a look at the three categorical SAT scores in my dataset.

In [12]:
individual_scores = ["SAT Math Avg. Score", "SAT Critical Reading Avg. Score","SAT Writing Avg. Score"]
data['sat_results'][individual_scores].head(3)

,SAT Math Avg. Score,SAT Critical Reading Avg. Score,SAT Writing Avg. Score
0,404,355,363
1,423,383,366
2,402,377,370


I will use a for loop to add each of the categorical SAT scores together in order to find the total SAT score.

In [13]:
data['sat_results']['sat_score'] = 0

for score in individual_scores:
    data["sat_results"][score] = pd.to_numeric(data["sat_results"][score], errors="coerce")
    data["sat_results"]["sat_score"] += data["sat_results"][score]

data["sat_results"]["sat_score"].head()

0    1122.0
1    1172.0
2    1149.0
3    1174.0
4    1207.0
Name: sat_score, dtype: float64

Next, I will find the latitude and longitude values by extracting them from the Location 1 column located within the hs_directory. Finding these coordiates will be useful for creating some of the maps I will design later. I will take a look at how the Location 1 column is formulated so that I have a better idea on how to extract the appropriate values from it.

In [14]:
data["hs_directory"]["Location 1"].head(3)

0    883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...
1    1110 Boston Road\nBronx, NY 10456\n(40.8276026...
2    1501 Jerome Avenue\nBronx, NY 10452\n(40.84241...
Name: Location 1, dtype: object

There are a pair of coordinates at the end of each data entry within the Location 1 column. I will use the re.findall function to find this pair of coordinates, which will be within a pair of parentheses.

In [15]:
re.findall("\(.+\)", data["hs_directory"]["Location 1"].iloc[0])

['(40.67029890700047, -73.96164787599963)']

Now that I have figured out how to extract the latitude and longitude from the Location 1, I will create a function that will select either the latitude or longitude from the Location 1 column so that I can create both a longitude and latitude column in the hs_directory dataset. Note that the lat_or_lon function rounds the latitudes and longitudes to the nearest ten thousandth's place, which is the precision most GPS systems use to denote a building (around 11 meters).

In [16]:
def lat_or_long(string, coord="lon"): # coord can be set to lon or lat, depending on which is needed (default=lon)
    coordinates = re.findall("\(.+\)", string)[0] # creates string with coordinates in format '(56.56, 56.56)'
    coordinates = coordinates.split(", ") # splits the two coordinates based on the comma separator
    
    # returns latitude or longitude w/o parenthesis and rounds to four digits (reasonable accuracy for GPS)
    if coord == "lat":
        return round(float(coordinates[0].replace("(", "")), 4)
    else:
        return round(float(coordinates[1].replace(")", "")), 4)

In [17]:
data["hs_directory"]["lat"] = data["hs_directory"]["Location 1"].apply(lat_or_long, args=(["lat"]))
data["hs_directory"]["lon"] = data["hs_directory"]["Location 1"].apply(lat_or_long)

data["hs_directory"]["lat"] = pd.to_numeric(data["hs_directory"]["lat"])
data["hs_directory"]["lon"] = pd.to_numeric(data["hs_directory"]["lon"])

data["hs_directory"][["lat", "lon"]].head(3)

,lat,lon
0,40.6703,-73.9616
1,40.8276,-73.9045
2,40.8424,-73.9162


In the next section, I will begin reducing various datasets into smaller subsets in order to make them easier to combine.

## Selecting Subsets of the Datasets

The first step in this section will be to reduce the class_size dataset to a subset that contains only high school programs and general ed programs. Additionally, I will find the mean of all classes within each school, thus significantly reducing the number of data entries in the dataset.

In [18]:
class_size_before = data["class_size"].shape
class_size_before

(27611, 17)

There are over 27,000 rows in the class_size dataset, so I will look at the columns of the class_size dataset to get a good idea on how to reduce the number of entries in the dataset.

In [19]:
data["class_size"].head(3)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
0,01,M,M015,P.S. 015 Roberto Clemente,0K,GEN ED,-,-,-,19.0,1.0,19.0,19.0,19.0,ATS,NaN,01M015
1,01,M,M015,P.S. 015 Roberto Clemente,0K,CTT,-,-,-,21.0,1.0,21.0,21.0,21.0,ATS,NaN,01M015
2,01,M,M015,P.S. 015 Roberto Clemente,01,GEN ED,-,-,-,17.0,1.0,17.0,17.0,17.0,ATS,NaN,01M015


The best columns to use to simplify the dataset are the GRADE and PROGRAM TYPE columns. Since I'm analyzing SAT scores, it only makes sense to analyze high school programs, so the GRADE column should indicate 9-12. Similarly, I should select whatever PROGRAM TYPE indicates the typical high school student, without looking at any special high school programs.

In [20]:
data["class_size"]["GRADE "].value_counts()

09-12      10644
MS Core     4762
0K-09       1384
0K          1237
01          1185
02          1167
03          1143
04          1140
05          1086
06           846
07           778
08           735
09            20
Name: GRADE , dtype: int64

In [21]:
data["class_size"]["PROGRAM TYPE"].value_counts()

GEN ED     14545
CTT         7460
SPEC ED     3653
G&T          469
Name: PROGRAM TYPE, dtype: int64

I will only select values of GRADE that equal "9-12" and only values of PROGRAM TYPE that equal "GEN ED". This should limit the number of data entries from 27K to around 6.5K since "9-12" consists of around half the values in GRADE and "GEN ED" consists of around half the values in PROGRAM TYPE.

In [22]:
class_size = data["class_size"]
class_size = class_size[class_size["GRADE "] == "09-12"]
class_size = class_size[class_size["PROGRAM TYPE"] == "GEN ED"]

class_size.head(3)

,CSD,BOROUGH,SCHOOL CODE,SCHOOL NAME,GRADE,PROGRAM TYPE,CORE SUBJECT (MS CORE and 9-12 ONLY),CORE COURSE (MS CORE and 9-12 ONLY),SERVICE CATEGORY(K-9* ONLY),NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,DATA SOURCE,SCHOOLWIDE PUPIL-TEACHER RATIO,DBN
225,01,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 9,-,63.0,3.0,21.0,19.0,25.0,STARS,NaN,01M292
226,01,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 10,-,79.0,3.0,26.3,24.0,31.0,STARS,NaN,01M292
227,01,M,M292,Henry Street School for International Studies,09-12,GEN ED,ENGLISH,English 11,-,38.0,2.0,19.0,16.0,22.0,STARS,NaN,01M292


In [23]:
class_size.shape

(6513, 17)

The number of rows in the class_size dataset decreased from over 27000 to around 6500, which is less than a quarter of its initial value. Next, I will examine how many unique "DBN" values there are (which indcate individual schools) so that I can find the mean values (number of students, average class size, etc.) for each school.

In [24]:
class_size["DBN"].value_counts()

09X505    21
20K505    21
24Q485    20
21K525    20
30Q445    20
          ..
26Q067     1
27Q183     1
09X303     1
30Q230     1
04M050     1
Name: DBN, Length: 583, dtype: int64

There are 583 unique high schools, so I next I will calculate the average for all values in the class_size dataset based on its DBN. Since the new dataset should only one value per DBN, the it should consist of 583 data entries.

In [25]:
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)

data["class_size"] = class_size
data["class_size"].head(3)

,DBN,NUMBER OF STUDENTS / SEATS FILLED,NUMBER OF SECTIONS,AVERAGE CLASS SIZE,SIZE OF SMALLEST CLASS,SIZE OF LARGEST CLASS,SCHOOLWIDE PUPIL-TEACHER RATIO
0,01M292,88.0,4.0,22.564286,18.5,26.571429,NaN
1,01M332,46.0,2.0,22.000000,21.0,23.500000,NaN
2,01M378,33.0,1.0,33.000000,33.0,33.000000,NaN


In [26]:
data["class_size"].shape

(583, 7)

As expected, the number of rows in the dataset equals the number of unique DBN's (high schools), so there are now only 583 rows in the dataset. Next, I will look at the demographics dataset in order to find a reasonable method of reducing the number of data entries.

In [27]:
demographics_before = data["demographics"].shape
data["demographics"].head(3)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158.0,56.2,123.0,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140.0,57.6,103.0,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143.0,54.8,118.0,45.2


Looking at the schoolyear column, there appears to be a data entry for every academic year that data was collected. To confirm this is the case, I will look at the value counts for the schoolyear column.

In [28]:
data["demographics"]["schoolyear"].value_counts()

20112012    1509
20102011    1498
20092010    1475
20082009    1441
20072008    1410
20062007    1386
20052006    1356
Name: schoolyear, dtype: int64

As expected, every academic year from 2005-2006 to 2011-2012 has around 1350-1500 values (the difference between the years is likely due to the opening/closing of schools). For my analysis, I will select only the most recent set of data entries, meaning those collected during the 2011-2012 academic year. Note that the academic year is stored as an integer (ex. 2011-2012 is stored as the integer 20112012).

In [29]:
data["demographics"] = data["demographics"][data["demographics"]["schoolyear"] == 20112012]
data["demographics"].head(3)

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,...,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,...,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3


Now that there is only one data entry for every school in the demographics dataset, I will begin looking at the graduation dataset and try to limit the number of data entries in that as well.

In [30]:
graduation_before = data["graduation"].shape
data["graduation"].head(3)

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
0,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2003,5,s,NaN,s,NaN,NaN,...,s,NaN,NaN,s,NaN,NaN,s,NaN,s,NaN
1,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2004,55,37,67.3,17,30.9,45.9,...,17,30.9,45.9,20,36.4,54.1,15,27.3,3,5.5
2,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2005,64,43,67.2,27,42.2,62.8,...,27,42.2,62.8,16,25.0,37.2,9,14.1,9,14.1


In a similar manner to the demographics dataset, the graduation dataset has a data entry for every academic year, which is denoted in the Cohort column. Thus, I can reduce the number of entries in the dataset by only selecting data entries from the most recent graduation year.

In [31]:
data["graduation"]["Cohort"].value_counts()

2006 Aug    4131
2006        4130
2005        3963
2004        3708
2003        3432
2002        3095
2001        2637
Name: Cohort, dtype: int64

I will only select data from the most recent graduation year, which is 2006. I will ignore the data entry containing data collected on 2006 Aug as it is likely very similar to the data collected in 2006 (note that August is the start of the academic year and that 2006 Aug has only one more data entry than 2006 does).

In [32]:
data["graduation"] = data["graduation"][data["graduation"]["Cohort"] == "2006"]

Next, I will take a look at the various possible values in the Demographic column to see if I can make a selection that will reduce the number of data entries in some manner.

In [33]:
data["graduation"]["Demographic"].value_counts()

Special Education Students     411
General Education Students     405
Total Cohort                   405
English Proficient Students    401
Female                         396
Hispanic                       395
Male                           395
Black                          394
English Language Learners      340
Asian                          296
White                          292
Name: Demographic, dtype: int64

I will select only values of the Demographic column that equal "Total Cohort" as these data entries contain statistical information from a population that consists of every type of person at the school. Although they have a separate data entry for all the different demographic categories, I want to focus my study on all high school students, so I will select from the entire student population.

In [34]:
data["graduation"] = data["graduation"][data["graduation"]["Demographic"] == "Total Cohort"]

Finally, I will take a look at the shape of the three datasets (class_size, demographics, and graduation) before and after I reduced the number of data entries so that I can see how significantly the datasets have changed.

In [35]:
class_size_after = data["class_size"].shape[0]
demographics_after = data["demographics"].shape[0]
graduation_after = data["graduation"].shape[0]


print("The class_size dataset has been reduced from {0} rows to {1} rows.".format(class_size_before[0], class_size_after))
print("The demographics dataset has been reduced from {0} rows to {1} rows.".format(demographics_before[0], demographics_after))
print("The graduation dataset has been reduced from {0} rows to {1} rows.".format(graduation_before[0], graduation_after))


The class_size dataset has been reduced from 27611 rows to 583 rows.
The demographics dataset has been reduced from 10075 rows to 1509 rows.
The graduation dataset has been reduced from 25096 rows to 405 rows.


All three datasets, especially class_size and graduation, have been significantly reduced in size by the data cleaning steps I applied. After I complete a few more tasks in the next section, the datasets will be ready to be combined into a single dataset.

# Combining the Datasets

Now that all of the datasets have been cleaned individually, I will take a look at the column names of all seven datasets in order to get a good idea of which column to merge the datasets on.

In [36]:
data.keys()

dict_keys(['ap_2010', 'class_size', 'demographics', 'graduation', 'hs_directory', 'sat_results', 'survey'])

In [37]:
data["ap_2010"].columns

Index(['DBN', 'SchoolName', 'AP Test Takers ', 'Total Exams Taken',
       'Number of Exams with scores 3 4 or 5'],
      dtype='object')

In [38]:
data["class_size"].columns

Index(['DBN', 'NUMBER OF STUDENTS / SEATS FILLED', 'NUMBER OF SECTIONS',
       'AVERAGE CLASS SIZE', 'SIZE OF SMALLEST CLASS', 'SIZE OF LARGEST CLASS',
       'SCHOOLWIDE PUPIL-TEACHER RATIO'],
      dtype='object')

In [39]:
data["demographics"].columns

Index(['DBN', 'Name', 'schoolyear', 'fl_percent', 'frl_percent',
       'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4',
       'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11',
       'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent',
       'ctt_num', 'selfcontained_num', 'asian_num', 'asian_per', 'black_num',
       'black_per', 'hispanic_num', 'hispanic_per', 'white_num', 'white_per',
       'male_num', 'male_per', 'female_num', 'female_per'],
      dtype='object')

In [40]:
data["graduation"].columns

Index(['Demographic', 'DBN', 'School Name', 'Cohort', 'Total Cohort',
       'Total Grads - n', 'Total Grads - % of cohort', 'Total Regents - n',
       'Total Regents - % of cohort', 'Total Regents - % of grads',
       'Advanced Regents - n', 'Advanced Regents - % of cohort',
       'Advanced Regents - % of grads', 'Regents w/o Advanced - n',
       'Regents w/o Advanced - % of cohort',
       'Regents w/o Advanced - % of grads', 'Local - n', 'Local - % of cohort',
       'Local - % of grads', 'Still Enrolled - n',
       'Still Enrolled - % of cohort', 'Dropped Out - n',
       'Dropped Out - % of cohort'],
      dtype='object')

In [41]:
data["hs_directory"].columns

Index(['DBN', 'school_name', 'boro', 'building_code', 'phone_number',
       'fax_number', 'grade_span_min', 'grade_span_max', 'expgrade_span_min',
       'expgrade_span_max', 'bus', 'subway', 'primary_address_line_1', 'city',
       'state_code', 'zip', 'website', 'total_students', 'campus_name',
       'school_type', 'overview_paragraph', 'program_highlights',
       'language_classes', 'advancedplacement_courses', 'online_ap_courses',
       'online_language_courses', 'extracurricular_activities',
       'psal_sports_boys', 'psal_sports_girls', 'psal_sports_coed',
       'school_sports', 'partner_cbo', 'partner_hospital', 'partner_highered',
       'partner_cultural', 'partner_nonprofit', 'partner_corporate',
       'partner_financial', 'partner_other', 'addtl_info1', 'addtl_info2',
       'start_time', 'end_time', 'se_services', 'ell_programs',
       'school_accessibility_description', 'number_programs', 'priority01',
       'priority02', 'priority03', 'priority04', 'priority05', 

In [42]:
data["sat_results"].columns

Index(['DBN', 'SCHOOL NAME', 'Num of SAT Test Takers',
       'SAT Critical Reading Avg. Score', 'SAT Math Avg. Score',
       'SAT Writing Avg. Score', 'sat_score'],
      dtype='object')

In [43]:
data["survey"].columns

Index(['DBN', 'rr_s', 'rr_t', 'rr_p', 'N_s', 'N_t', 'N_p', 'saf_p_11',
       'com_p_11', 'eng_p_11', 'aca_p_11', 'saf_t_11', 'com_t_11', 'eng_t_11',
       'aca_t_11', 'saf_s_11', 'com_s_11', 'eng_s_11', 'aca_s_11',
       'saf_tot_11', 'com_tot_11', 'eng_tot_11', 'aca_tot_11'],
      dtype='object')

Note that all of seven of the datasets have a DBN column, so I will merge the datasets on that column. I will set my combined dataset to sat_results and then merge it with the following datasets as so:
- Left Merge = used for the ap\_2010 and graduation datasets
- Inner Merge = used for the class\_size, demographics, survey, and hs\_directory datasets
\end{itemize}

In [44]:
combined = data["sat_results"]
combined = combined.merge(data["ap_2010"], how="left", on="DBN")
combined = combined.merge(data["graduation"], how="left", on="DBN")

In [45]:
to_merge = ["class_size", "demographics", "survey", "hs_directory"]

for m in to_merge:
    combined = combined.merge(data[m], on="DBN", how="inner")
    
combined.columns

Index(['DBN', 'SCHOOL NAME', 'Num of SAT Test Takers',
       'SAT Critical Reading Avg. Score', 'SAT Math Avg. Score',
       'SAT Writing Avg. Score', 'sat_score', 'SchoolName', 'AP Test Takers ',
       'Total Exams Taken',
       ...
       'priority04', 'priority05', 'priority06', 'priority07', 'priority08',
       'priority09', 'priority10', 'Location 1', 'lat', 'lon'],
      dtype='object', length=157)

Even though the datasets are merged, note that there are 157 columns. Before I begin my analysis, I will need to get rid of most of the columns.

# Data Cleaning Post-Merge

Even though the datasets have been merged, there are a few modifications to the dataset that will need to be completed before it is ready for analysis:
- Create a district column by by selecting the first two digits from the DBN column
- Select and reorder the columns that will be useful to my analysis
- Rename some of the columns so that they all fit the same naming convention
- Replace all NaN values within a column with the column mean to prevent bugs
- Change the number types of the values in certain columns
- Save the new cleaned, combined dataset to a csv file

First, I will create a function that will select the first to digits of a string. I will apply this function to the DBN column in order to create a new District column. This column will be useful if I decide to analyze by district in Part 2 of this project.

In [46]:
def first_two(string):
    return string[:2]

combined["District"] = combined["DBN"].apply(first_two)

Next, I will select only the columns that will be useful for my analysis, which include those below. Note how I also rearranged the order of the columns to make it easier to read the tables.

In [47]:
useful_columns = ["DBN", "District", "SCHOOL NAME", "sat_score",
                  "Num of SAT Test Takers", "AP Test Takers ", "Total Exams Taken", "total_enrollment",
                  "AVERAGE CLASS SIZE", "male_per", "white_per", "black_per",
                  "hispanic_per", "asian_per", "ell_percent", "saf_t_11", 
                  "saf_s_11", "lat", "lon"]

combined = combined[useful_columns]

Next, I will rename most of the column names that remain so they follow the same naming convention:
- Capitalization of the first letter in every word
- Spaces (rather than underscores) to denote multiple words
- Changing a few of the names to make them more easily understandable in a table format

In [48]:
column_names = {
                "SCHOOL NAME": "School",
                "sat_score": "SAT Score",
                "Num of SAT Test Takers": "SAT Test Takers",
                "AP Test Takers ": "AP Test Takers",
                "Total Exams Taken": "AP Tests Taken",
                "total_enrollment": "Total Enrollment",
                "AVERAGE CLASS SIZE": "Class Size",
                "male_per": "Percent Male",
                "white_per": "Percent White",
                "black_per": "Percent Black",
                "asian_per": "Percent Asian",
                "hispanic_per": "Percent Hispanic",
                "ell_percent": "Percent ELL",
                "saf_t_11": "Teacher Safety",
                "saf_s_11": "Student Safety",
                "lat": "Lat",
                "lon": "Lon"
               }

combined = combined.rename(columns=column_names)
combined.head(3)

,DBN,District,School,SAT Score,SAT Test Takers,AP Test Takers,AP Tests Taken,Total Enrollment,Class Size,Percent Male,Percent White,Percent Black,Percent Hispanic,Percent Asian,Percent ELL,Teacher Safety,Student Safety,Lat,Lon
0,01M292,01,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,1122.0,29,NaN,NaN,422,22.564286,61.4,1.7,29.1,53.8,14.0,22.3,6.3,6.0,40.7138,-73.9853
1,01M448,01,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,91,39.0,49.0,394,22.231250,57.4,2.3,22.6,45.9,29.2,21.1,6.6,6.0,40.7123,-73.9848
2,01M450,01,EAST SIDE COMMUNITY SCHOOL,1149.0,70,19.0,21.0,598,21.200000,54.7,10.4,23.9,55.4,9.7,5.0,7.3,NaN,40.7298,-73.9830


Note that some of the columns have NaN values, which could later result in systematic errors if I decide to manipulate the values in each column. For example, in the next step, I am planning on rounding the values in the Class Size column to the nearest tenth's place. If there is a NaN value in one of the data entries for that column, it will result in an error since NaN is not a float. Therefore, I will replace all of the NaN values with the means for each of the columns. For example, if the mean AP Tests Taken is 55, I will replace all NaN values in the AP Tests Taken column with the number 55. If I can't calculate a mean for that column, I will replace the value with a zero.

In [49]:
combined = combined.fillna(combined.mean())
combined = combined.fillna(0)

combined.head(3)

,DBN,District,School,SAT Score,SAT Test Takers,AP Test Takers,AP Tests Taken,Total Enrollment,Class Size,Percent Male,Percent White,Percent Black,Percent Hispanic,Percent Asian,Percent ELL,Teacher Safety,Student Safety,Lat,Lon
0,01M292,01,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,1122.0,29,129.028846,197.038462,422,22.564286,61.4,1.7,29.1,53.8,14.0,22.3,6.3,6.000000,40.7138,-73.9853
1,01M448,01,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172.0,91,39.000000,49.000000,394,22.231250,57.4,2.3,22.6,45.9,29.2,21.1,6.6,6.000000,40.7123,-73.9848
2,01M450,01,EAST SIDE COMMUNITY SCHOOL,1149.0,70,19.000000,21.000000,598,21.200000,54.7,10.4,23.9,55.4,9.7,5.0,7.3,6.611667,40.7298,-73.9830


The numeric values in the SAT Score, SAT Test Takers, and AP Tests Taken should all be integers, since the original values were integers and thus I must maintain the same levels of precision for all value. Similarly, the Class Size and the Student Safety column should be rounded to the tenth's column to match the rounding convention of all the other floats in the dataset. The Latitude and Longitude columns do not need to change as maintaining it's current level of precision will be necessary in order to map out the different locations in Part 2 of this project.

In [50]:
int_change = ["SAT Score", "AP Test Takers", "AP Tests Taken"]
round_change = ["Class Size", "Student Safety"]

combined[int_change] = combined[int_change].astype(int)
combined[round_change] = combined[round_change].round(1)
combined.head(3)

,DBN,District,School,SAT Score,SAT Test Takers,AP Test Takers,AP Tests Taken,Total Enrollment,Class Size,Percent Male,Percent White,Percent Black,Percent Hispanic,Percent Asian,Percent ELL,Teacher Safety,Student Safety,Lat,Lon
0,01M292,01,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,1122,29,129,197,422,22.6,61.4,1.7,29.1,53.8,14.0,22.3,6.3,6.0,40.7138,-73.9853
1,01M448,01,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,1172,91,39,49,394,22.2,57.4,2.3,22.6,45.9,29.2,21.1,6.6,6.0,40.7123,-73.9848
2,01M450,01,EAST SIDE COMMUNITY SCHOOL,1149,70,19,21,598,21.2,54.7,10.4,23.9,55.4,9.7,5.0,7.3,6.6,40.7298,-73.9830


Now that the data cleaning section of this project is complete, I will save the new dataset to a new CSV file so that it can be easily referenced in Part 2, which is the analysis portion of the project.

In [51]:
combined.to_csv('combined.csv', index=False)